# Init

In [1]:
using LinearAlgebra
using NearestNeighbors
using StaticArrays

# Types
#### $m$: Dimensionality of each point (rows)  
#### $n$: Total number of points (cols)

In [3]:
"""
Container struct of points, their values, bookkeeping, and spatial trees
mutable: Partially initialized and filled in later
"""
mutable struct QStore
    pnts::Matrix{ Float64 }
    vals::Array{ Float64 }
    mDim
    nPts
    kdTree
    blTree
    _DEBUG 
    QStore( mDim, nPts ) = (
        x = new(); # Create an object for partial initialization
        x.pnts   = zeros(Float64, mDim, nPts); # Interpolation anchors
        x.vals   = zeros(Float64,       nPts); # Values at anchors
        x.mDim   = mDim; # --------------------- Dimensionality of anchors 
        x.nPts   = nPts; # --------------------- Number of anchors
        x.kdTree = nothing; # ------------------ kD-Tree storage
        x.blTree = nothing; # ------------------ Ball Tree storage
        x._DEBUG = true; # --------------------- Debug Flag
        if x._DEBUG
            println( "Created `QStore` with ", x.mDim, " dimensions and ", x.nPts, " points!" );
        end;
        return x
    );
end

QStore

In [4]:
function fetch_by_indices( qSto::QStore, idxList )
    N = size( idxList, 1 )
    p = []
    v = []
    for i = 1:N
        append!( p, qSto.pnts )
        append!( v, qSto.vals )
    end
    return p, v
end

fetch_by_indices (generic function with 1 method)

In [5]:
"""
Use `truthFunc` to load the correct values for each anchor in `qSto`
NOTE: This function assumes that `truthFunc` has the same `mDim` as `truthFunc`
"""
function load_QStore_with_true_values( qSto::QStore, truthFunc )
    # For every column of stored points, compute the truth value
    for j in 1:qSto.nPts
        qSto.vals[j] = truthFunc( qSto.pnts[:,j] )
    end
end

load_QStore_with_true_values

In [8]:
include("utils.jl") 

sample_bbox_N

In [9]:
"""
Hyperplane with a uniform positive slope along all axes
"""
function get_nD_example_plane_eq( dims, slope )
    factor = slope / dims
    Y = repeat( [factor], dims )
    return function( X )
        dot( X, Y )
    end
end

"""
Cone of `height` that slopes downward from origin 
"""
function get_nD_example_cone_eq( height, slope )
    return function( X )
        height - norm( X ) * slope
    end
end

"""
Hill of `height` that slopes downward from origin 
"""
function get_nD_example_hill_eq( height, scale )
    return function( X )
        height * exp( -norm(X)/scale )
    end
end

f1 = get_nD_example_plane_eq( 2, 1 )
f2 = get_nD_example_cone_eq( 5, 1 )
f3 = get_nD_example_hill_eq( 5, 2.5 )

#5 (generic function with 1 method)

In [11]:
_DIMS = 2
mesh  = regular_grid_pts_nD( [[-10,+10],[-10,+10]], [1,1] );
qGrid = QStore( _DIMS, size( mesh )[2] );
qGrid.pnts = copy( mesh );
qGrid.load_QStore_with_true_values( f1 )

Created `QStore` with 2 dimensions and 441 points!


LoadError: type QStore has no field load_QStore_with_true_values

## Conclusion
#### Julia does not use dot.notation as in `<object>.<method>`